# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv('cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
y_val = housing['medianHouseValue']

In [5]:
x_vals = housing.drop('medianHouseValue',axis=1)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_test_split

<function sklearn.model_selection._split.train_test_split>

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x_vals, y_val, test_size=0.3, random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler_model = MinMaxScaler()

In [11]:
scaler_model

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
scaler_model.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [13]:
# columns and index are important because they give us the right headers
X_train = pd.DataFrame(data=scaler_model.transform(X_train),
             columns=X_train.columns,
             index=X_train.index)

In [14]:
X_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
6761,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
3010,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
7812,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
8480,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
1051,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631


In [15]:
X_test = pd.DataFrame(data=scaler_model.transform(X_test),
             columns=X_test.columns,
             index=X_test.index)

In [16]:
X_test.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
16086,0.686275,0.046264,0.045158,0.025873,0.048841,0.353133
8816,0.705882,0.027417,0.020795,0.012709,0.023187,0.770182
7175,0.901961,0.032326,0.040813,0.041662,0.042592,0.133626
16714,0.313725,0.043212,0.046089,0.032840,0.048018,0.263576
14491,0.411765,0.088433,0.069367,0.043728,0.072192,0.660046


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [17]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [18]:
import tensorflow as tf

In [19]:
# feature columns are only the columns on the x axis (house value is on y)
med_age = tf.feature_column.numeric_column('housingMedianAge')
tot_rooms = tf.feature_column.numeric_column('totalRooms')
tot_brooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
med_income = tf.feature_column.numeric_column('medianIncome')

In [20]:
feat_cols = [med_age, tot_rooms, tot_brooms, pop, households, med_income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [21]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,
                                                 batch_size=10,
                                                 num_epochs=1000,
                                                 shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [22]:
dnn_model = tf.estimator.DNNRegressor(hidden_units=[10,10,10],
                                       feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_session_config': None, '_model_dir': '/var/folders/d2/0mftypjn6lj4_6tsb6y_1y_r0000gn/T/tmp385kcs31', '_keep_checkpoint_max': 5, '_tf_random_seed': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [23]:
dnn_model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/d2/0mftypjn6lj4_6tsb6y_1y_r0000gn/T/tmp385kcs31/model.ckpt.
INFO:tensorflow:step = 1, loss = 5.30865e+11
INFO:tensorflow:global_step/sec: 627.821
INFO:tensorflow:step = 101, loss = 6.36535e+11 (0.160 sec)
INFO:tensorflow:global_step/sec: 704.633
INFO:tensorflow:step = 201, loss = 4.62533e+11 (0.143 sec)
INFO:tensorflow:global_step/sec: 661.918
INFO:tensorflow:step = 301, loss = 4.66946e+11 (0.150 sec)
INFO:tensorflow:global_step/sec: 644.446
INFO:tensorflow:step = 401, loss = 5.62283e+11 (0.155 sec)
INFO:tensorflow:global_step/sec: 624.544
INFO:tensorflow:step = 501, loss = 4.83825e+11 (0.160 sec)
INFO:tensorflow:global_step/sec: 654.961
INFO:tensorflow:step = 601, loss = 1.19692e+11 (0.153 sec)
INFO:tensorflow:global_step/sec: 673.441
INFO:tensorflow:step = 701, loss = 1.76248e+11 (0.150 sec)
INFO:tensorflow:global_step/sec: 648.883
INFO:tensorflow:step = 801, loss = 1.21916e+11 (0.1

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [24]:
# note: predict fn uses test data not training data!
input_fn_predict = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,
                                                 batch_size=10,
                                                 num_epochs=1000,
                                                 shuffle=False)

In [25]:
pred_gen = dnn_model.predict(input_fn_predict)

In [26]:
predictions = list(pred_gen)
predictions

INFO:tensorflow:Restoring parameters from /var/folders/d2/0mftypjn6lj4_6tsb6y_1y_r0000gn/T/tmp385kcs31/model.ckpt-1000


[{'predictions': array([ 206895.21875], dtype=float32)},
 {'predictions': array([ 240370.265625], dtype=float32)},
 {'predictions': array([ 205367.15625], dtype=float32)},
 {'predictions': array([ 164904.], dtype=float32)},
 {'predictions': array([ 219834.21875], dtype=float32)},
 {'predictions': array([ 187986.171875], dtype=float32)},
 {'predictions': array([ 213746.5], dtype=float32)},
 {'predictions': array([ 190940.], dtype=float32)},
 {'predictions': array([ 189126.140625], dtype=float32)},
 {'predictions': array([ 149174.234375], dtype=float32)},
 {'predictions': array([ 191545.6875], dtype=float32)},
 {'predictions': array([ 213238.390625], dtype=float32)},
 {'predictions': array([ 178731.3125], dtype=float32)},
 {'predictions': array([ 162783.03125], dtype=float32)},
 {'predictions': array([ 236880.109375], dtype=float32)},
 {'predictions': array([ 156298.5], dtype=float32)},
 {'predictions': array([ 189670.203125], dtype=float32)},
 {'predictions': array([ 167885.75], dtype=f

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [28]:
final_predictions = []
for pred in predictions:
    final_predictions.append(pred['predictions'])

In [38]:
from sklearn.metrics import mean_squared_error

In [45]:
mean_squared_error(y_test, final_predictions)

ValueError: Found input variables with inconsistent numbers of samples: [6192, 6192000]

# Great Job!